### Notes 

T5 Paper: https://arxiv.org/pdf/1910.10683.pdf

T5 Tokenizer: https://github.com/huggingface/transformers/blob/master/src/transformers/tokenization_t5.py

Important Tasks: https://docs.google.com/document/d/1weIZM6QTlnitpPQmpg-WeV2RW70TnYmDuogBQPr5mB0/edit

In [8]:
#James Chartouni
#Joey Park
#Raef Khan

import torch
import pandas as pd
import numpy as np
import pickle
import os, io, glob

import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data import DataLoader

from torchtext.datasets.translation import TranslationDataset
from torchtext.data import Field, BucketIterator

import sentencepiece as spm

from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config, T5Model


from spacy.tokenizer import Tokenizer
from spacy.vocab import Vocab

from sklearn.model_selection import train_test_split

from utils import *


In [56]:
VOCAB_SIZE = 5000

In [9]:
ls data/AD_NMT-master

LAV-MSA-2-both.pkl        Magribi_MSA-train.pkl     english-arabic-test.pkl
LAV-MSA-2-test.pkl        README.md                 lav_formatted_.txt
LAV-MSA-2-train.pkl       en_formatted_.txt         mag_formatted_.txt
Magribi_MSA-both.pkl      english-Arabic-both.pkl   msa_formatted_.txt
Magribi_MSA-test.pkl      english-Arabic-train.pkl


In [10]:
file_path = 'data/AD_NMT-master/'

with open(file_path + "english-Arabic-train.pkl", 'rb') as handle:
    data_English_MSA_trainval = pickle.load(handle)

with open(file_path + "english-Arabic-test.pkl", 'rb') as handle:
    data_English_MSA_test = pickle.load(handle)

with open(file_path + "english-Arabic-both.pkl", 'rb') as handle:
    data_English_MSA_both = pickle.load(handle) 




with open(file_path + "LAV-MSA-2-train.pkl", 'rb') as handle:
    data_LAV_MSA_trainval = pickle.load(handle) 

with open(file_path + "LAV-MSA-2-test.pkl", 'rb') as handle:
    data_LAV_MSA_test = pickle.load(handle) 

with open(file_path + "LAV-MSA-2-both.pkl", 'rb') as handle:
    data_LAV_MSA_both = pickle.load(handle) 




with open(file_path + "Magribi_MSA-train.pkl", 'rb') as handle:
    data_Magribi_MSA_trainval = pickle.load(handle) 
    
with open(file_path + "Magribi_MSA-test.pkl", 'rb') as handle:
    data_Magribi_MSA_test = pickle.load(handle) 

with open(file_path + "Magribi_MSA-both.pkl", 'rb') as handle:
    data_Magribi_MSA_both = pickle.load(handle) 
    

In [11]:
print(data_English_MSA_both[0:5])
print(data_LAV_MSA_both[0:5])
print(len(data_English_MSA_trainval))
print(len(data_English_MSA_both))

[['Tom was also there', 'كان توم هنا ايضا'], ['That old woman lives by herself', 'تلك المراة العجوز تسكن بمفردها'], ['He went abroad for the purpose of studying English', 'سافر خارج البلد ليتعلم الانجليزية'], ['There is a fork missing', 'هناك شوكة ناقصة'], ["I don't know this game", 'لا اعرف هذه اللعبة']]
[['لا انا بعرف وحدة راحت ع فرنسا و معا شنتا حطت فيها الفرش', 'لا اعرف واحدة ذهبت الى فرنسا و لها غرفة و ضعت فيها الافرشة'], ['روح بوشك و فتول عاليسار', 'اذهب تقدم و استدر يسارا'], ['لا لا لازم انه يكون عندك موضوع ما في اشي', ' لا لا يجب ان يكون لديك موضوع هذا ضروري'], ['اوعي تبعدي من هون بلاش تضيعي ', 'لا تبتعد عن هنا حتى لا تفقد الطريق '], ['قصدي صراحة يما انا كمان كرهته من يوم ما عملتيه زي ما بتعمله خالتي كرهته و صرت ما باطيقه بالمرة', 'اقصد صراحة يا امي انا ايضا كرهته من يوم حضرته مثلما تحضره خالتي كرهته و اصبحت لا اطيقه ابدا']]
9000
10001


## Prepare Datasets

example: https://iwslt2010.fbk.eu/node/32/

We need to take our training and test sets from the pkl files and create new .txt files that are formatted so that the standard torchtext Dataset class can read them

Data format:
each line consists of three fields divided by the character '\'
sentences consisting of words divided by single spaces
format: <SENTENCE_ID>\<PARAPHRASE_ID>\<TEXT>
Field_1: sentence ID
Field_2: paraphrase ID
Field_3: MT develop sentence / reference translation
Text input example:
DEV_001\01\This is the first develop sentence.
DEV_002\01\This is the second develop sentence.
Reference translation example:
DEV_001\01\1st reference translation for 1st input
DEV_001\02\2nd reference translation for 1st input
...
DEV_002\01\1st reference translation for 2nd input
DEV_002\02\2nd reference translation for 2nd input
...
Languages:
Arabic-English
CSTAR03 testset: 506 sentences, 16 reference translations
IWSLT04 testset: 500 sentences, 16 reference translations
IWSLT05 testset: 506 sentences, 16 reference translations
IWSLT07 testset: 489 sentences, 6 reference translations
IWSLT08 testset: 507 sentences, 16 reference translations
French-English
CSTAR03 testset: 506 sentences, 16 reference translations
IWSLT04 testset: 500 sentences, 16 reference translations
IWSLT05 testset: 506 sentences, 16 reference translations
Turkish-English
CSTAR03 testset: 506 sentences, 16 reference translations
IWSLT04 testset: 500 sentences, 16 reference translations


In [12]:
ls data/AD_NMT-master/

LAV-MSA-2-both.pkl        Magribi_MSA-train.pkl     english-arabic-test.pkl
LAV-MSA-2-test.pkl        README.md                 lav_formatted_.txt
LAV-MSA-2-train.pkl       en_formatted_.txt         mag_formatted_.txt
Magribi_MSA-both.pkl      english-Arabic-both.pkl   msa_formatted_.txt
Magribi_MSA-test.pkl      english-Arabic-train.pkl


In [13]:
#splits the train dataset into train and validation sets, define test set as datafile
eng_msa_train, eng_msa_val = train_test_split(data_English_MSA_trainval, test_size=.2)
eng_msa_test = data_English_MSA_test

lav_msa_train, lav_msa_val = train_test_split(data_LAV_MSA_trainval, test_size=.2)
lav_msa_test = data_LAV_MSA_test

mag_msa_train, mag_msa_val = train_test_split(data_Magribi_MSA_trainval, test_size=.2)
mag_msa_test = data_Magribi_MSA_test

In [14]:
print(len(eng_msa_train))
print(len(eng_msa_val))

print(len(lav_msa_train))
print(len(lav_msa_val))

print(len(mag_msa_train))
print(len(mag_msa_val))

7200
1800
11044
2761
14188
3548


In [15]:
file_path = 'data/'

def pytorch_format(ds, src='en', trg='msa', datatype=''):
    src_formatted = datatype + '_' + src + '_' + trg + '.' + src
    trg_formatted = datatype + '_' + src + '_' + trg + '.' + trg
    
    with open(file_path + datatype + "/" + src_formatted, 'wt') as srctxt, open(file_path + datatype + "/" + trg_formatted, 'wt') as trgtxt:
        for i, arr in enumerate(ds):
            srctxt.write(datatype.upper() + '_' + str(i).zfill( len(str(len(ds))) - len(str(i))) + '\\01\\' + arr[0] + '\n')
            trgtxt.write(datatype.upper() + '_' + str(i).zfill( len(str(len(ds))) - len(str(i))) + '\\01\\' + arr[1] + '\n')

In [16]:
#splits each language pair file into datasets of single language, to be merged again by the pytorch dataset class later

pytorch_format(eng_msa_train, 'eng', 'msa', 'train')
pytorch_format(eng_msa_val, 'eng', 'msa', 'val')
pytorch_format(eng_msa_test, 'eng', 'msa', 'test')

pytorch_format(lav_msa_train, 'lav', 'msa', 'train')
pytorch_format(lav_msa_val, 'lav', 'msa', 'val')
pytorch_format(lav_msa_test, 'lav', 'msa', 'test')

pytorch_format(mag_msa_train, 'mag', 'msa', 'train')
pytorch_format(mag_msa_val, 'mag', 'msa', 'val')
pytorch_format(mag_msa_test, 'mag', 'msa', 'test')

In [17]:
cat data/train/train_eng_msa.eng

TRAIN_000\01\I have not finished my homework yet
TRAIN_001\01\Were they busy last week?
TRAIN_002\01\I dislike being alone
TRAIN_003\01\Did you know my father?
TRAIN_004\01\I want to go to America someday
TRAIN_005\01\What's your occupation?
TRAIN_006\01\He was very busy all day
TRAIN_007\01\It's fifty kilometers to Paris
TRAIN_008\01\Summer has ended
TRAIN_009\01\Would you please pay attention?
TRAIN_10\01\She bore the pain bravely
TRAIN_11\01\The guard was on duty all night
TRAIN_12\01\Get out
TRAIN_13\01\He is very kind
TRAIN_14\01\My son is now as tall as I am
TRAIN_15\01\Is it new?
TRAIN_16\01\You can say whatever you want
TRAIN_17\01\I wanted to buy a painting from Tom
TRAIN_18\01\You'll need a temporary bridge
TRAIN_19\01\Many tourists visit Kyoto in the spring
TRAIN_20\01\I am coming today
TRAIN_21\01\She took her own life
TRAIN_22\01\I'd like to see you again
TRAIN_23\01\What time will you leave?
TRAIN_24\01\Are you alone?
TRAIN_25\01\I added this website to my favorites
TRAIN

TRAIN_4177\01\It's exactly what I wanted
TRAIN_4178\01\It was probably not true
TRAIN_4179\01\Awesome!
TRAIN_4180\01\I do not like science
TRAIN_4181\01\We are hungry
TRAIN_4182\01\She didn't know what to do with the problem
TRAIN_4183\01\When did you begin studying English?
TRAIN_4184\01\I think I mistakenly deleted that file
TRAIN_4185\01\We haven't seen you in the past four years
TRAIN_4186\01\Try to improve your English
TRAIN_4187\01\I had my car stolen last night
TRAIN_4188\01\There are mothers and fathers who will lie awake after the children fall asleep and wonder how they'll make the mortgage or pay their doctor's bills or save enough for their child's college education
TRAIN_4189\01\When does the bank open?
TRAIN_4190\01\Mary wanted to know if Tom had a girlfriend
TRAIN_4191\01\Do you have a room of your own?
TRAIN_4192\01\She doesn't need to work
TRAIN_4193\01\I'm not tired
TRAIN_4194\01\How tall you are!
TRAIN_4195\01\Tomorrow I'll buy what I need

TRAIN_4197\01\We really en

In [18]:
cat data/train/train_eng_msa.msa

TRAIN_000\01\لم انه واجبي بعد
TRAIN_001\01\هل كانوا مشغولين في الاسبوع الماضي؟
TRAIN_002\01\لا احب ان اكون وحدي
TRAIN_003\01\اكنت تعرف والدي؟
TRAIN_004\01\اريد ان اذهب الى امريكا في يوم من الايام
TRAIN_005\01\ما مهنتك؟
TRAIN_006\01\كان مشغولا طوال اليوم
TRAIN_007\01\تبعد عن باريس خمسين كيلومترا
TRAIN_008\01\ولى فصل الصيف
TRAIN_009\01\هلا اصغيتم رجاء؟
TRAIN_10\01\تحملت الالم بشجاعة
TRAIN_11\01\حرس الحارس الليل بطوله
TRAIN_12\01\اخرج!
TRAIN_13\01\انه لطيف جدا
TRAIN_14\01\ابني بنفس طولي حاليا
TRAIN_15\01\هل هذا جديد؟
TRAIN_16\01\يمكنك ان تقول ما تريد
TRAIN_17\01\اردت شراء لوحة من توم
TRAIN_18\01\ستحتاج جسرا مؤقتا
TRAIN_19\01\يزور العديد من السياح مدينة كيوتو في الربيع 
TRAIN_20\01\انا قادم اليوم
TRAIN_21\01\قامت بالانتحار
TRAIN_22\01\اود رؤيتك مجددا
TRAIN_23\01\متى ستغادر؟
TRAIN_24\01\هل انت وحدك؟
TRAIN_25\01\لقد وضعت هذا الموقع في مفضلاتي
TRAIN_26\01\انا اشك بروايته 
TRAIN_27\01\امتاك من انك تعرف كيف تفعل هذا؟
TRAIN_28\01\يمكن للبجعة ان تستوعب الكثير من الاسماك في منقارها
TRAIN_29\01\ما 

TRAIN_3851\01\استانف قراءة الكتاب
TRAIN_3852\01\خط امي جميل
TRAIN_3853\01\كم رائع انت!
TRAIN_3854\01\ايجب ان اقوم بذلك على الفور؟
TRAIN_3855\01\انشئت مدارس
TRAIN_3856\01\استمتع بوقتك
TRAIN_3857\01\من فضلك تعال
TRAIN_3858\01\افعل ما يخبرك توم به لتفعله وكل شئ سيكون بخير
TRAIN_3859\01\سباحة توم سريعة جدا
TRAIN_3860\01\اذهب الى الكنيسة مع والديك وصل لها
TRAIN_3861\01\اننا نستهلك الكثير من الماء يوميا
TRAIN_3862\01\وصلت منذ لحظات
TRAIN_3863\01\ماذا افعل بالسكين؟
TRAIN_3864\01\لم يعد لدينا سكر
TRAIN_3865\01\احضر اولادك معك
TRAIN_3866\01\ قدمي ما زالت تؤلمني
TRAIN_3867\01\احدهم سرق محفظتي
TRAIN_3868\01\الباب مفتوح
TRAIN_3869\01\لم اعد اذكر ما حدث
TRAIN_3870\01\قارورة مياه  من فضلك
TRAIN_3871\01\اتهمتني باني كذبت
TRAIN_3872\01\انني قلق عليه
TRAIN_3873\01\انظر الى هذا!
TRAIN_3874\01\هذه ملاحظة في غاية الاهمية
TRAIN_3875\01\دعونا نصنع تجارة
TRAIN_3876\01\لم اخبر توم اني اعرفك
TRAIN_3877\01\لوح توم
TRAIN_3878\01\توم يحتاج الى محام جيد
TRAIN_3879\01\يجب عليك ان تدرس بجد اكثر
TRAIN_3880\01\انت يمك

In [19]:
cat data/train/train_lav_msa.lav

TRAIN_0000\01\بتقولي لا انا بحكي عن هذه بس من شدة غباء الملاحظات اللي حطيتها فيها
TRAIN_0001\01\يا رب ما تخلي اهدافهم تصيب حتى العارضة
TRAIN_0002\01\المفروض في تشرين الاول
TRAIN_0003\01\منيح  في شقاوة
TRAIN_0004\01\سليمة انت بتنقلي المدونة بانتظام 
TRAIN_0005\01\اه
TRAIN_0006\01\و صار يقولهم اضربوهم قاتلوهم
TRAIN_0007\01\ما في مشكلة بهيك
TRAIN_0008\01\ومفروض منا زيادة توعيتهم
TRAIN_0009\01\يا ويلك لو تحكي معها بصراحة مهي زي القرة
TRAIN_010\01\مهو كمان مفش ملك فقير
TRAIN_011\01\شوفت هذا كتير عنجد
TRAIN_012\01\احيت قضيتها
TRAIN_013\01\على فكرت لقد اكدت موضوع القرض العقاري مع الاستاذة و انت لما تفضي اعطينا مثال عن قرض بيت او محل قصدي كل الحالات الموجودة
TRAIN_014\01\عند هداك الاستاذ
TRAIN_015\01\الواضح انو هو كان معو حق 
TRAIN_016\01\و خوخة هيها عند فاطمة
TRAIN_017\01\قلتلها شغلات متل هي ما بتقول عنها الناس بس بيقولوا ماشاللله تبارك الله بقولوا مستحيل
TRAIN_018\01\مختص في اشعة الاعصاب
TRAIN_019\01\هاد يلي قدمك هلق ما في حدا بيحب امه قده
TRAIN_020\01\هذه هي المصيبة انهم يتبعوا بس و مش عارف

TRAIN_4924\01\حتى الدردشة فينا نتصل بالنهار لما بيكون في وقت احيانا بيصير معي المسا انو لاقي وقت
TRAIN_4925\01\و حتى انهم كانو يقضوا ايام بحالها في السفر على رجليهم عشان يوصلوا لمكان معين
TRAIN_4926\01\وما في مشاكل للصوت رح تواجهك مع البرنامج هاد
TRAIN_4927\01\جمال الحوريات في الاساطير 
TRAIN_4928\01\انا طالب ماجستير في الفيزيا وبدي اطار نظري 
TRAIN_4929\01\وينا ليكا
TRAIN_4930\01\بينطفي كل دقيقتين
TRAIN_4931\01\فهذا يطلب بحث و التاني يعطيك تمرين و غيره ينبهك للتحضير للواجب او الاستجواب وغيره بطلب منك الحفظ
TRAIN_4932\01\ليش
TRAIN_4933\01\ومخبييين حالهم من راسهم لحد رجلهم
TRAIN_4934\01\اسمه شاي باللبن 
TRAIN_4935\01\رقم البيت
TRAIN_4936\01\عشان هو بيلعب شوي بالالوان
TRAIN_4937\01\لا ما زودوهم  مراد
TRAIN_4938\01\هناك المصاري
TRAIN_4939\01\افضل انو نحكي كل يوم عن مجال محدد
TRAIN_4940\01\اذا زعجني المدير باضرب
TRAIN_4941\01\شوفنا الاعلان تبع نتائج السداسي الاول
TRAIN_4942\01\كمان انت تخصصك تسيير عندك مواد حفظ كتير
TRAIN_4943\01\مين كان عندها هناك
TRAIN_4944\01\مشي الحال لكن مو انت
TRAIN_

TRAIN_9875\01\بضل شوي بعدين ��روح
TRAIN_9876\01\خبريها
TRAIN_9877\01\قالتلي اخت شمام كيف حالك قلت بخير و الحمد لله
TRAIN_9878\01\عطاني المؤطر شوية شغل لاعمله جمعت الوثائق بس لسا ما بديت
TRAIN_9879\01\طيب مو مشكلة
TRAIN_9880\01\بهالدنيا ما في شي بيدوم من غير سلاحك
TRAIN_9881\01\هي هيك بتقولي بتمنى اكله من كتر روعته
TRAIN_9882\01\بتهيالي تاخر الوقت
TRAIN_9883\01\اي اذا كنت مشغول عملها بالليل و انا رح اتصل فيك المسا
TRAIN_9884\01\ليش هو شو بيشتغل
TRAIN_9885\01\انت كمان صرخت في وجهها
TRAIN_9886\01\لالا اكبار و اعمول اللي بدك ياه
TRAIN_9887\01\عنا البرق قوي
TRAIN_9888\01\احنا ممكن نضل او  او مع بعض
TRAIN_9889\01\خلينا نحاول انا ما نغلطش في الكتابة
TRAIN_9890\01\كل واحد يحكي لحاله و الا حتصير فوضى
TRAIN_9891\01\مرة اتنين مساطيل قاعدين على القهوة فواحد عميقول للتاني 
TRAIN_9892\01\وانت الي الك الاختيار في الاول والاخير 
TRAIN_9893\01\هذا كل اللي عنا في اشي تاني
TRAIN_9894\01\تقريبا  بس لما عبقري زيي يوقف قدام حيوان زيك 
TRAIN_9895\01\فلما بتفاجئو بانهم بتعبو وبدعو عليهم 
TRAIN_9896\01\كلشي فر

In [20]:
cat data/train/train_lav_msa.msa

TRAIN_0000\01\تقول لي لا انا اتكلم عن هذه فقط من شدة غباء الملاحظات التي اضعها فيها
TRAIN_0001\01\يارب لا تدع اهدافهم تصيب حتى العارضة
TRAIN_0002\01\المفروض في اكتوبر
TRAIN_0003\01\على ما يرام في شقاوة
TRAIN_0004\01\سليمة هل تنقلين المدونة بانتظام
TRAIN_0005\01\اجل
TRAIN_0006\01\ و صار يقول لهم اضربوهم قاتلوهم   
TRAIN_0007\01\لا مشكلة في ذلك
TRAIN_0008\01\ويجب علينا زياده توعيتهم
TRAIN_0009\01\وان تكلمها بالصدق يا ويلك  تكون كالبقرة 
TRAIN_010\01\وايضا لا يوجد  ملك  فقير 
TRAIN_011\01\ارايت هذا كثير حقا
TRAIN_012\01\احيت قضيتها
TRAIN_013\01\على فكرة لقد اكدت امر القرض العقاري مع الاستاذ و انت عندما تفرغين اعطنا مثالا عن قرض منزل او محل  يعني كل الحالات الموجودة
TRAIN_014\01\عند ذلك الاستاذ
TRAIN_015\01\بيتضح  انه  كان  على  حق
TRAIN_016\01\و خوخة   هل هي عند فاطمة
TRAIN_017\01\ قلت لها امور كهذه لا يقال عنها فقط بل يقال ما شاء الله تبارك الله يقال مستحيل   
TRAIN_018\01\ مختص في اشعة الاعصاب   
TRAIN_019\01\ هذا الذي امامك الان لا يوجد شخص في الدنيا يحب امه مثله    
TRAIN_020\01\تلك ه

TRAIN_4284\01\سيصبح العمل فرديا
TRAIN_4285\01\ اكملوا حديثكم كيف قضيتم العطلة   
TRAIN_4286\01\احدى الفتيات بالامس
TRAIN_4287\01\ هذا امر طبيعي احتمال ان تقوم انت باشياء اضافية   
TRAIN_4288\01\تتطلب القليل من العمل و لكن تحوي كلاما
TRAIN_4289\01\وايضا  الاخ   تويتى   من  وقف  على  منتصف  السلم    انتظر الى  التاسع  فى  ذالك  الكتاب    الذى  لم  نبادر  بشراءه حتى الان 
TRAIN_4290\01\غدا ايضا امتحانان التربية المدنية في الصباح و في المساء التربية البدنية
TRAIN_4291\01\كان لدي اثنان لكني ارسلت لك الاول و لم ارسل الثاني
TRAIN_4292\01\اتعلمين انه اقصر رئيس حاليا
TRAIN_4293\01\ قلت له ساقول لك شيئا لو حملت حاسوبا نقالا و جبت الصيدليات هل استطيع ان احصل عليهم ام لا   
TRAIN_4294\01\لكن فيما يخص طلبة الطب فمعه حق
TRAIN_4295\01\هناك امور كثيرة
TRAIN_4296\01\الامور تسير على خطى البلد
TRAIN_4297\01\بالعربية هو بطيئ
TRAIN_4298\01\نعم
TRAIN_4299\01\لا اعلم
TRAIN_4300\01\اتذكرين ذلك الالم الذي اصابني
TRAIN_4301\01\تستوجب المساعدة
TRAIN_4302\01\قلت لك انني والدك
TRAIN_4303\01\اردت ان اريكم شيئا مزعج

TRAIN_8244\01\اما انا فقد كان ابن عمي يابى ان ياتي الى بيتنا ابدا
TRAIN_8245\01\هو طلب مهنم تحضيره على شكل تطبيقات
TRAIN_8246\01\نعم نحن انهينا تقريبا
TRAIN_8247\01\ابداوا بالحديث
TRAIN_8248\01\هذا محمول لبيبة
TRAIN_8249\01\و كانت اكبر متعة وقتها لما نذهب للحدائق مع امي و نلعب
TRAIN_8250\01\ومنه فريق في المجلة ثم نرى ذلك
TRAIN_8251\01\ لا هو لا يدرسه في الجامعة   
TRAIN_8252\01\ماذا هذا الذى فعلته 
TRAIN_8253\01\ساظل اكتب و انا واقفة
TRAIN_8254\01\مثلا على الثانية و النصف
TRAIN_8255\01\و لا يستطيع التعبير عن هذا بالكلمات
TRAIN_8256\01\نعم يقال انه يدهشك حتى تنظر اليه
TRAIN_8257\01\قال لها جمال تعطي كل واحد منا مئتي مليون 
TRAIN_8258\01\جيد
TRAIN_8259\01\اذكر ثم عاودت الذهاب من الجهة الاخرى و سلمت عليه مرة ثانية فضحك معي
TRAIN_8260\01\اصبحت الازمة في كل مكان حتى اوربا لم تعد تصلح للهجرة
TRAIN_8261\01\على رسلك يا بسمة  الم تكوني صغيرة في الماضي
TRAIN_8262\01\ ماذا تفعلين في حياتك ماذا تدرسين   
TRAIN_8263\01\انتظرت كثيرا ليتصلوا   على التاسعة اخرجت شريحة نجمة الخاصة بي و اتصلت بامي فاتصل

## Build Vocabulary 

Sentence Piece Google Colab
https://github.com/google/sentencepiece/blob/master/python/sentencepiece_python_module_example.ipynb

In [21]:
!mkdir data/vocab
!mkdir data/model

mkdir: data/vocab: File exists
mkdir: data/model: File exists


In [22]:
"""
Testing:
Create a text file with all the vocab available from all sources for each language for SentencePiece to create a library 

TODO: implement bpemb model for MSA vocab/train SPM on larger datasets for dialects
"""

en_vocab = open("data/vocab/eng_vocab.txt", "wt")
msa_vocab = open("data/vocab/msa_vocab.txt", "wt")
lav_vocab = open("data/vocab/lav_vocab.txt", "wt")
mag_vocab = open("data/vocab/mag_vocab.txt", "wt")

MSA_text = ""
EN_text = ""

def create_vocab(file='', src='en_vocab', tgt='msa_vocab'):
  for line in file:
        src_sent = line[0]
        src_words = src_sent.split(" ")
        for count, word in enumerate(src_words):
            src.write(word)
        src.write("\n")
        
        tgt_sent = line[1]
        tgt_words = tgt_sent.split(" ")
        for count, word in enumerate(tgt_words):
            tgt.write(word)
        tgt.write("\n")

create_vocab(data_English_MSA_both, en_vocab, msa_vocab)
create_vocab(data_LAV_MSA_both, lav_vocab, msa_vocab)
create_vocab(data_Magribi_MSA_both, mag_vocab, msa_vocab)

en_vocab.close()
msa_vocab.close()
lav_vocab.close()
mag_vocab.close()

In [58]:
spm.SentencePieceTrainer.train('--input=data/vocab/eng_vocab.txt --model_prefix=data/model/eng --vocab_size=' + str(VOCAB_SIZE))
spm.SentencePieceTrainer.train('--input=data/vocab/msa_vocab.txt --model_prefix=data/model/msa --vocab_size=' + str(VOCAB_SIZE))
spm.SentencePieceTrainer.train('--input=data/vocab/lav_vocab.txt --model_prefix=data/model/lav --vocab_size=' + str(VOCAB_SIZE))
spm.SentencePieceTrainer.train('--input=data/vocab/mag_vocab.txt --model_prefix=data/model/mag --vocab_size=' + str(VOCAB_SIZE))
#

True

In [59]:
ls data/model

eng.model  lav.model  mag.model  msa.model
eng.vocab  lav.vocab  mag.vocab  msa.vocab


In [60]:
sp = spm.SentencePieceProcessor()
sp.load('data/model/eng.model')

True

In [61]:
print(sp.encode_as_pieces('This is a test'))
print(sp.encode_as_ids('This is a test'))

['▁This', '▁', 'is', '▁', 'a', '▁', 't', 'est']
[67, 47, 10, 47, 9, 47, 7, 649]


## Spacy Tokenizer

In [62]:
eng_vocab_data = open("data/model/eng.vocab", "r")
msa_vocab_data = open("data/model/msa.vocab", "r")
lav_vocab_data = open("data/model/lav.vocab", "r")
mag_vocab_data = open("data/model/mag.vocab", "r")

eng_vocab_list = []
msa_vocab_list = []
lav_vocab_list = []
mag_vocab_list = []

for line in eng_vocab_data.readlines():
    eng_vocab_list.append(line.split("\t")[0])

for line in msa_vocab_data.readlines():
    msa_vocab_list.append(line.split("\t")[0])

for line in lav_vocab_data.readlines():
    lav_vocab_list.append(line.split("\t")[0])

for line in mag_vocab_data.readlines():
    mag_vocab_list.append(line.split("\t")[0])


In [63]:
# msa_vocab_list[0:200]

In [64]:
en_vocab = Vocab(strings=eng_vocab_list)
spacy_en_tokenizer = Tokenizer(en_vocab)

msa_vocab = Vocab(strings=msa_vocab_list)
spacy_msa_tokenizer = Tokenizer(msa_vocab)

lav_vocab = Vocab(strings=lav_vocab_list)
spacy_lav_tokenizer = Tokenizer(lav_vocab)

mag_vocab = Vocab(strings=mag_vocab_list)
spacy_mag_tokenizer = Tokenizer(mag_vocab)

## TF Tokenizer

https://huggingface.co/transformers/model_doc/t5.html#t5tokenizer

In [65]:
ls data/model

eng.model  lav.model  mag.model  msa.model
eng.vocab  lav.vocab  mag.vocab  msa.vocab


In [66]:
tokenizer = T5Tokenizer('data/model/eng.model')
tokenizer.batch_encode_plus(data_English_MSA_both)
msa_tokenizer = T5Tokenizer("data/model/msa.model")
en_tokenizer = T5Tokenizer("data/model/eng.model")

In [67]:
input_ids = msa_tokenizer.encode('هل بامكاني الدخول؟ </s>', return_tensors='pt')

In [68]:
input_ids

tensor([[  86,    3,  270,  273,    5,   51, 2110,   15,    3,    2]])

In [69]:
#decode to make sure you can go back and forth between the encoding properly 
#@raef can you verify the Arabic?
output = msa_tokenizer.decode(input_ids[0])
print(output)

هل بامكاني الدخول؟ 


## Pytorch Data Set and Data Loader

https://github.com/google-research/text-to-text-transfer-transformer

pytorch dataset: https://pytorch.org/text/_modules/torchtext/datasets/translation.html

pytorch dataset documentation: https://torchtext.readthedocs.io/en/latest/datasets.html#iwslt

example dataset: https://iwslt2010.fbk.eu/node/32/

Field API: https://pytorch.org/text/data.html#torchtext.data.Field
Field Source: https://pytorch.org/text/_modules/torchtext/data/field.html#Field

https://pytorch.org/text/_modules/torchtext/data/utils.html#get_tokenizer
    
we need to pass our tokenizer as a function     

In [70]:
SRC = Field(tokenize = spacy_en_tokenizer,
            init_token = '<sos>',
            eos_token = '<eos>',
            lower = False)

TRG = Field(tokenize = spacy_msa_tokenizer,
            init_token = '<sos>',
            eos_token = '<eos>',
            lower = False)

# SRC = Field(tokenize = spacy_lav_tokenizer,
#             init_token = '<sos>',
#             eos_token = '<eos>',
#             lower = False)

# SRC = Field(tokenize = spacy_mag_tokenizer,
#             init_token = '<sos>',
#             eos_token = '<eos>',
#             lower = False)

In [71]:
eng_msa_train_dataset = TranslationDataset(path='data/train/train_eng_msa.', exts=('eng', 'msa'), fields=(SRC, TRG))
#lav_msa_train_dataset = TranslationDataset(path='data/train/train_lav_msa.', exts=('lav', 'msa'), fields=(SRC, TRG))
#mag_msa_train_dataset = TranslationDataset(path='data/train/train_mag_msa.', exts=('mag', 'msa'), fields=(SRC, TRG))

In [72]:
SRC.build_vocab(eng_msa_train_dataset)
TRG.build_vocab(eng_msa_train_dataset)
#SRC.build_vocab(lav_msa_train_dataset)
#TRG.build_vocab(lav_msa_train_dataset)
#SRC.build_vocab(mag_msa_train_dataset)
#TRG.build_vocab(mag_msa_train_dataset)

In [73]:
ex = eng_msa_train_dataset[0]

In [74]:
print(ex.src) 
print(ex.trg)

TRAIN_000\01\I have not finished my homework yet
TRAIN_000\01\لم انه واجبي بعد


In [75]:
ls data/train

en_formatted_.txt   train_eng_msa.eng   train_lav_msa.msa
lav_formatted_.txt  train_eng_msa.msa   train_mag_msa.mag
msa_formatted_.txt  train_lav_msa.lav   train_mag_msa.msa


In [76]:
#train, validation and test are probably wrong.  
#https://github.com/pytorch/text/blob/master/torchtext/datasets/translation.py

'''
Arguments:
            exts: A tuple containing the extension to path for each language.
            fields: A tuple containing the fields that will be used for data
                in each language.
            path (str): Common prefix of the splits' file paths, or None to use
                the result of cls.download(root).
            root: Root dataset storage directory. Default is '.data'.
            train: The prefix of the train data. Default: 'train'.
            validation: The prefix of the validation data. Default: 'val'.
            test: The prefix of the test data. Default: 'test'.
            Remaining keyword arguments: Passed to the splits method of
                Dataset.

'''


train_data, valid_data, test_data = eng_msa_train_dataset.splits(path= 'data/', train='train/train_eng_msa', validation='val/val_eng_msa', test='test/test_eng_msa', exts=('.eng', '.msa'),
                                                    fields = (SRC, TRG))

In [77]:
BATCH_SIZE =32

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size = BATCH_SIZE,
    device = device)

## Model Training

https://huggingface.co/transformers/model_doc/t5.html#training

https://pytorch.org/tutorials/beginner/torchtext_translation_tutorial.html

In [78]:
count = 0

for x in train_iterator:
    print(x)
    print(x.src)
    print(x.trg)
    print(spacy_en_tokenizer.decode(x.src))
    count += 1
    
print(count)


[torchtext.data.batch.Batch of size 32]
	[.src]:[torch.LongTensor of size 15x32]
	[.trg]:[torch.LongTensor of size 12x32]
tensor([[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
         2, 2, 2, 2, 2, 2, 2, 2],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0,
         0, 3, 0, 0, 0, 0, 0, 0],
        [0, 0, 3, 0, 3, 0, 0, 0, 0, 1, 1, 0, 0, 3, 0, 0, 0, 3, 0, 0, 0, 3, 1, 0,
         0, 1, 0, 3, 0, 0, 3, 3],
        [0, 0, 1, 0, 1, 0, 0, 0, 3, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0,
         3, 1, 3, 1, 0, 0, 1, 1],
        [0, 0, 1, 0, 1, 0, 0, 3, 1, 1, 1, 3, 0, 1, 0, 3, 0, 1, 3, 3, 0, 

AttributeError: 'spacy.tokenizer.Tokenizer' object has no attribute 'decode'

In [ ]:
PAD_IDX = TRG.vocab.stoi['<pad>']

criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)

In [ ]:
def train(model: nn.Module,
          iterator: BucketIterator,
          optimizer: optim.Optimizer,
          criterion: nn.Module,
          clip: float):

    model.train()

    epoch_loss = 0

    for _, batch in enumerate(iterator):

        src = batch.src
        trg = batch.trg

        optimizer.zero_grad()

        output = model(src, trg)

        output = output[1:].view(-1, output.shape[-1])
        trg = trg[1:].view(-1)

        loss = criterion(output, trg)

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(iterator)


In [ ]:
def evaluate(model: nn.Module,
             iterator: BucketIterator,
             criterion: nn.Module):

    model.eval()

    epoch_loss = 0

    with torch.no_grad():

        for _, batch in enumerate(iterator):

            src = batch.src
            trg = batch.trg

            output = model(src, trg, 0) #turn off teacher forcing?

            output = output[1:].view(-1, output.shape[-1])
            trg = trg[1:].view(-1)

            loss = criterion(output, trg)

            epoch_loss += loss.item()

    return epoch_loss / len(iterator)

In [54]:
#declare model
T5Config(vocab_size=VOCAB_SIZE)



T5Config {
  "_num_labels": 2,
  "architectures": null,
  "bad_words_ids": null,
  "bos_token_id": null,
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": null,
  "do_sample": false,
  "dropout_rate": 0.1,
  "early_stopping": false,
  "eos_token_id": 1,
  "finetuning_task": null,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_factor": 1.0,
  "is_decoder": false,
  "is_encoder_decoder": true,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_epsilon": 1e-06,
  "length_penalty": 1.0,
  "max_length": 20,
  "min_length": 0,
  "model_type": "t5",
  "n_positions": 512,
  "no_repeat_ngram_size": 0,
  "num_beams": 1,
  "num_heads": 8,
  "num_layers": 6,
  "num_return_sequences": 1,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pad_token_id": 0,
  "prefix": null,
  "pruned_heads": {},
  "relative_attention_num_buckets": 32,
  "repetition_penalty": 1.0,
  "task_specific_params": n

In [52]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()

    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

NameError: name 'model' is not defined

In [4]:
def trainIters(model, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    optimizer = AdamW.SGD(model.parameters(), lr=learning_rate)

    criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX) #CHECK IF THE BEST LOSS FUNCTION

    for iter in range(1, n_iters + 1):
        

        loss = train(decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [ ]:
model = 
n_iteres = 10
trainIters(model, n_iters)

## Model Inference

https://github.com/huggingface/transformers/blob/master/examples/translation/t5/evaluate_wmt.py

In [ ]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i : i + n]

In [ ]:
from tqdm import tqdm
def generate_translations(lns, output_file_path, model_size, batch_size, device):
    model = T5ForConditionalGeneration.from_pretrained(model_size)
    model.to(device)
    #returned to using T5 tokenizer
    tokenizer = en_tokenizer

    task_specific_params = model.config.task_specific_params

    with Path(output_file_path).open("w") as output_file:
        for batch in tqdm(list(chunks(lns, batch_size))):
            batch = [text for text in batch]
            
            dct = tokenizer.batch_encode_plus(batch, max_length=512, return_tensors="pt", pad_to_max_length=True)

            input_ids = dct["input_ids"].to(device)
            attention_mask = dct["attention_mask"].to(device)

            translations = model.generate(input_ids=input_ids, attention_mask=attention_mask)
            dec = [
                tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in translations
            ]

            for hypothesis in dec:
                output_file.write(hypothesis + "\n")

In [ ]:
from pathlib import Path
def calculate_bleu_score(output_lns, refs_lns, score_path):
    bleu = corpus_bleu(output_lns, [refs_lns])
    result = "BLEU score: {}".format(bleu.score)
    with Path(score_path).open("w") as score_file:
        score_file.write(result)

In [ ]:
!mkdir data/output/
!touch data/output/translated.msa
!touch data/output/score_eng_msa.txt

In [ ]:
#got rid of the argparse and just put it in as params
def run_generate(model_size='t5-base', input_path='data/train/train_eng_msa.eng', output_path='data/output/translated.msa', ref_path='data/train/train_eng_msa.eng', score_path='data/output/score_eng_msa.txt', batch_size=BATCH_SIZE):
    device = torch.device("cuda" if torch.cuda.is_available() and not args.no_cuda else "cpu")

    # Read input lines into python
    with open(input_path, "r") as input_file:
        input_lns = [x.strip() for x in input_file.readlines()]

    generate_translations(input_lns, output_path, model_size, batch_size, device)

    # Read generated lines into python
    with open(output_path, "r") as output_file:
        output_lns = [x.strip() for x in output_file.readlines()]

    # Read reference lines into python
    with open(ref_path, "r") as reference_file:
        refs_lns = [x.strip() for x in reference_file.readlines()]

    calculate_bleu_score(output_lns, refs_lns, args.score_path)

In [ ]:
run_generate()